# Thermal Evolution Model for Calcite Recrystallisation and Organic Matter 

### Setting up the CoNus Library

First of all we need to set up CoNus with Almond within the Jupyter Notebook.

In [1]:
//Using the CoNus library with Jupyter Notebook with Almond kernal, add the following resolver:
interp.repositories() ++= Seq(coursierapi.MavenRepository.of(
"https://jitpack.io"
))

In [2]:
//import the CoNus library
import $ivy. `org.carbonateresearch::conus:0.2.3`

import $ivy.$                                    

In [3]:
//Importing stepped models
import org.carbonateresearch.conus._ //wildcard import
import org.carbonateresearch.conus.modelzoo.GeneralGeology._ //general geology model
import org.carbonateresearch.conus.modelzoo.PasseyHenkesClumpedDiffusionModel._ //solid state diffusion model

import org.carbonateresearch.conus._ //wildcard import

import org.carbonateresearch.conus.modelzoo.GeneralGeology._ //general geology model

import org.carbonateresearch.conus.modelzoo.PasseyHenkesClumpedDiffusionModel._ //solid state diffusion model

In [4]:
//For the Almond kernal, create an Almond Simulator
val sim = new AlmondSimulator

16:46:02.958 [CoNuS-akka.actor.default-dispatcher-3] INFO akka.event.slf4j.Slf4jLogger - Slf4jLogger started


SLF4J: A number (1) of logging calls during the initialization phase have been intercepted and are
SLF4J: now being replayed. These are subject to the filtering rules of the underlying logging system.
SLF4J: See also http://www.slf4j.org/codes.html#replay


sim: AlmondSimulator = org.carbonateresearch.conus.AlmondSimulator@6326bb96

### CoNus Forward Model Set Up

Below is a step by step layout of the model set up for this forward model. Model is looking to understand temepratures reached during burial and how calcite recrystallisation versus organic matter transofrmation records them.

In [5]:
// Constant variables
val rhocal:Double = 2.71 //Density of carbonates
val rhofluid = 1.029 //Density of seawater
val cOfluid = 889000 //concentration of Oxygen in fluid
val cCfluid = 29 //concentration of Carbon in fluid
val cOcalcite = 18000 //480000 //concentration of Oxygen in stoichiometric calcite
val cCcalcite = 120000 //concentration of Carbon in stoichiometric calcite
val A = 3.012E13 //frequency factor of type II organic matter (Burnham & Sweeney, 1989)
val Ea = 220000 //Activation Energy of organic matter (Qigui et al., 2010)
val R = 8.31 //gas constant
val Poro = 0.1 //porosity of formation

rhocal: Double = 2.71
rhofluid: Double = 1.029
cOfluid: Int = 889000
cCfluid: Int = 29
cOcalcite: Int = 18000
cCcalcite: Int = 120000
A: Double = 3.012E13
Ea: Int = 220000
R: Double = 8.31
Poro: Double = 0.1

In [6]:
// setting model variables
val initialAge:ModelVariable[Double] = ModelVariable("Initial age",97.0,"Ma")
val finalAge:ModelVariable[Double] = ModelVariable("Final age",0.0,"Ma")
val fractionRecrystallisation:ModelVariable[Double] = ModelVariable("Fraction of Recrystallisation", 0.1,"")
val fractionFluidExchange:ModelVariable[Double] = ModelVariable ("Fraction of Fluid Exchange (Mixing of fluids)", 0.01, "")
val weightFractionFluid:ModelVariable[Double] = ModelVariable("Weight fraction of fluid",0.01,"")
val d18OfluidMixed:ModelVariable[Double] = ModelVariable("Fluid Oxygen composition from mixing of fluids due to fluid exchange", -4.0,"‰")
val d18OfluidAfterDissolution:ModelVariable[Double] = ModelVariable("Fluid Oxygen composition after dissolution", -4.0,"‰")
val d18OcalciteNewPhase: ModelVariable[Double] = ModelVariable("Oxygen Composition of the new phase of calcite", -6.0,"‰") //calculated from fluid (-4) using kim equation
val d18OfluidAfterReprecipitation: ModelVariable[Double] = ModelVariable("Fluid Oxygen composition after reprecipitation of calcite", -4.0,"‰")
val d18OcalciteBulkFinal: ModelVariable[Double] = ModelVariable("Oxygen composition of the bulk rock", -6.0, "‰") //calculated from fluid (-4) using kim equation
val d13CfluidMixed:ModelVariable[Double] = ModelVariable("Fluid Carbon composition from mixing of fluids due to fluid exchange", -3.0,"‰")
val d13CfluidAfterDissolution:ModelVariable[Double] = ModelVariable("Fluid Carbon composition after dissolution", -3.0,"‰")
val d13CcalciteNewPhase: ModelVariable[Double] = ModelVariable("Carbon Composition of the new phase of calcite", 0.5,"‰")
val d13CfluidAfterReprecipitation:ModelVariable[Double] = ModelVariable("Fluid carbon composition after reprecipitation of calcite", -3.0,"‰")
val d13CcalciteBulkFinal:ModelVariable[Double] = ModelVariable("Carbon composition of the bulk rock", 0.5, "‰")
val D47rec:ModelVariable[Double] = ModelVariable("D47 with partial recrystallization",.59,"‰")
val sampleTempRecrystallised:ModelVariable[Double] = ModelVariable("Recrystallisation Temperature",25,"˚C")
val initalBurialAtModelStart:ModelVariable[Double] = ModelVariable("Initial burial at model start",0.0,"meters")
val d18OfluidBackCalculated:ModelVariable[Double] = ModelVariable("Back calculated fluid composition",-4.0, "‰ ")
val reactionRateOrganics:ModelVariable[Double] = ModelVariable("Organic reaction rate",3.28E-26) 
val apparentTempOrganics:ModelVariable[Double] = ModelVariable("Apparent organic temperature",25,"˚C")
val maxBurialDepth:ModelVariable[Double] = ModelVariable("Maximum Burial Depth",0, "m")

Variable Initial age with initial value of 97.0 Ma defined

Variable Final age with initial value of 0.0 Ma defined

Variable Fraction of Recrystallisation with initial value of 0.1 defined

Variable Fraction of Fluid Exchange (Mixing of fluids) with initial value of 0.01 defined

Variable Weight fraction of fluid with initial value of 0.01 defined

...

Variable Initial burial at model start with initial value of 0.0 meters defined

Variable Back calculated fluid composition with initial value of -4.0 ‰ defined

Variable Organic reaction rate with initial value of 3.28E-26 defined

Variable Apparent organic temperature with initial value of 25.0 ˚C defined

Variable Maximum Burial Depth with initial value of 0.0 m defined

In [37]:
// Initialise model conditions as lists
val burialHistory = List((97.0,0.0), (62.0, 4000.0), (0.0,-70.0))
val geothermalGradientHistory= List((97.0, 21.84),(0.0, 30.0)) 
val surfaceTemperaturesHistory = List((97.0,25.0),(0.0,20.6))
val numberOfSteps = 140
val ageList:List[Double] = List(97.0)
val finalAgeList:List[Double] = List(0.0)
val fractionRecrystallisationList:List[Double] = List(0.1)

burialHistory: List[(Double, Double)] = List(
  (97.0, 0.0),
  (62.0, 4000.0),
  (0.0, -70.0)
)
geothermalGradientHistory: List[(Double, Double)] = List(
  (97.0, 21.84),
  (0.0, 30.0)
)
surfaceTemperaturesHistory: List[(Double, Double)] = List(
  (97.0, 25.0),
  (0.0, 20.6)
)
numberOfSteps: Int = 140
ageList: List[Double] = List(97.0)
finalAgeList: List[Double] = List(0.0)
fractionRecrystallisationList: List[Double] = List(0.1)

### Equations inputted into model
#### Carbon and Oxygen composition of fluid based on fluid exchange takes the mixed fluid (original fluid) from the previous step and the fraction of the fluid after dissolution (final fluid) from the previous step.

In [38]:
//Fluid composition based of fluid exchange - prior to any dissolution of carbonate
import math._
//Oxygen composition - based on fluid exchange
val d18OfluidMixedFun = (s:Step) => (d18OfluidMixed(s-1) * (1 - fractionFluidExchange(s)) + d18OfluidAfterReprecipitation(s-1) * fractionFluidExchange(s))

//Carbon composition - based on fluid exchange
val d13CfluidMixedFun = (s:Step) => (d13CfluidMixed(s-1) * (1 - fractionFluidExchange(s)) + d13CfluidAfterReprecipitation(s-1) * fractionFluidExchange(s))

import math._
//Oxygen composition - based on fluid exchange

d18OfluidMixedFun: Step => Double = ammonite.$sess.cmd37$Helper$$Lambda$3297/0x00000008015ae318@4b05e762
d13CfluidMixedFun: Step => Double = ammonite.$sess.cmd37$Helper$$Lambda$3298/0x00000008015ae6e8@45062898

#### Carbon and Oxygen composition of fluid based on the amount of interaction between the rock and the fluid. Equations taken from Banner & Hanson (1990).

**cOsystem** - concentration of oxygen in the system

**cCsystem** - concentraction of carbon in the system


In [39]:
//Fluid composition based on the amount of carbonate dissolution
//Oxygen composition for fluid based on water-rock interactions during dissolution
val d18OfluidAfterDissolutionFun = (s:Step) => {
    val cOsystem = (weightFractionFluid(s) * cOfluid) + ((1 - weightFractionFluid(s)) * cOcalcite)
    (((d18OfluidMixed(s) * weightFractionFluid(s) * cOfluid) + (d18OcalciteBulkFinal(s-1) * (1 - weightFractionFluid(s)) * cOcalcite)) / cOsystem)    
}

//carbon composition for fluid based on water-rock interactions during dissolution
val d13CfluidAfterDissolutionFun = (s:Step) => { 
    val cCsystem = (weightFractionFluid(s) * cCfluid) + ((1 - weightFractionFluid(s)) * cCcalcite)
    (((d13CfluidMixed(s) * weightFractionFluid(s) * cCfluid) + (d13CcalciteBulkFinal(s-1) * (1 - weightFractionFluid(s)) * cCcalcite)) / cCsystem)
}

d18OfluidAfterDissolutionFun: Step => Double = ammonite.$sess.cmd38$Helper$$Lambda$3305/0x00000008015b0668@69452874
d13CfluidAfterDissolutionFun: Step => Double = ammonite.$sess.cmd38$Helper$$Lambda$3306/0x00000008015b0a38@f2806ae

#### Carbon and Oxygen composition of the new phase of calcite that will precipitate based on the composition of the fluid within the system and the composition of the calcite which dissolved in the previous step. 

The below equations assume equilibrium, therefore fraction of recrystallisation has not yet been applied. This would be the new composition of the calcite and this would be equate to the new bulk composition if 100% recrystallisation (complete) has occured.

Oxygen composition of the new phase of calcite is based on the **Kim et al. (1997)** equation. 

Carbon composition of the new phase of calcite is based on **Banner & Hanson (1990)** equation.

In [40]:
//Bulk composition of the new phase of calcite that has precipitated
//Oxygen composition of the new recrystallised phase of calcite
val d18OcalciteNewPhaseFun = (s:Step) => (18.03 * (pow(10,3) * pow((sampleTempRecrystallised(s) + 273.15), -1)) - 32.42 + (d18OfluidAfterDissolution(s)/1.03091-30.86))

//Carbon composition of the new recrystallised phase of calcite
val d13CcalciteNewPhaseFun = (s:Step) => { 
    val cCsystem = (weightFractionFluid(s) * cCfluid) + ((1 - weightFractionFluid(s)) * cCcalcite)
    val alpha = math.exp((-2.4612+(7666.3/100) - (2.9880*pow(10,3)/pow((sampleTempRecrystallised(s)+273.15),6)))/1000)
    val d13Csystem = ((d13CfluidAfterDissolution(s) * cCfluid * weightFractionFluid(s)) + (d13CcalciteBulkFinal(s-1) * cCcalcite * (1-weightFractionFluid(s))))/cCsystem
    ((d13Csystem * cCsystem * alpha) - (1000 * cCfluid * weightFractionFluid(s) * (1-alpha)))/((cCcalcite * (1-weightFractionFluid(s))*alpha)+cCfluid*weightFractionFluid(s))
}

d18OcalciteNewPhaseFun: Step => Double = ammonite.$sess.cmd39$Helper$$Lambda$3313/0x00000008015b28d0@6400813b
d13CcalciteNewPhaseFun: Step => Double = ammonite.$sess.cmd39$Helper$$Lambda$3314/0x00000008015b2ca0@843a777

#### Oxygen and Carbon composition of the fluid once the new phase of calcite has precipitated. This is the fluid that remains once calcite has reprecipitated after dissolution.

Oxygen composition of the fluid uses the **Kim et al. (1997)** equation as the fluid composition will be based on the calcite that precipitated. 

Carbon composition of the fluid uses the **Banner and Hanson (1990)** equation. 

In [41]:
//Fluid composition after recrystallisation (complete/partially)
//Oxygen compositon for fluid based on new phase of carbonate
val d18OfluidAfterReprecipitationFun = (s:Step) => ((d18OcalciteNewPhase(s)*1.03091+30.86)+32.42)/(18.03*pow(10,3)*pow((sampleTempRecrystallised(s)+273.15),-1)) * fractionRecrystallisation(s) + (d18OfluidAfterReprecipitation(s-1) * (1 - fractionRecrystallisation(s)))

//carbon composition based on carbon within the system after recrystallisation
val d13CfluidAfterReprecipitationFun = (s:Step) => {
    val fracNewCalcite = fractionRecrystallisation(s) * (1 - weightFractionFluid(s))
    (d13CfluidAfterDissolution(s) - (d13CcalciteNewPhase(s) * (fracNewCalcite/(cCcalcite + fracNewCalcite))))
 }


d18OfluidAfterReprecipitationFun: Step => Double = ammonite.$sess.cmd40$Helper$$Lambda$3321/0x00000008015b4b38@509736c9
d13CfluidAfterReprecipitationFun: Step => Double = ammonite.$sess.cmd40$Helper$$Lambda$3322/0x00000008015b4f08@7dcbbfc4

#### Bulk composition of the calcite based on how much of the new phase makes up the new bulk rock utilising the fraction of recrystallisation. 

These equations take the composition of the new phase that will have reprecipitated plus how much of the original bulk rock remains prior to recrystallisation.

In [42]:
//bulk composition of calcite based on the amount of recrystallisation
//oxygen composition
val d18OcalciteBulkFinalFun = (s:Step) => (fractionRecrystallisation(s) * d18OcalciteNewPhase(s)) + ((1 - fractionRecrystallisation(s)) * d18OcalciteBulkFinal(s-1))

//carbon composition
val d13CcalciteBulkFinalFun = (s:Step) => (fractionRecrystallisation(s) * d13CcalciteNewPhase(s)) + ((1 - fractionRecrystallisation(s)) * d13CcalciteBulkFinal(s-1))

d18OcalciteBulkFinalFun: Step => Double = ammonite.$sess.cmd41$Helper$$Lambda$3329/0x00000008015b6da0@18bf19df
d13CcalciteBulkFinalFun: Step => Double = ammonite.$sess.cmd41$Helper$$Lambda$3330/0x00000008015b7170@7758e0b5

#### Fluid composition calcuated from my raw measured data

This is based on the calcite composition, which is affected by recrystallisation, therefore is not the true compsoiton of the fluid it is an enriched value. A final back calculated fluid composition is done using **Kim et al., (1997)**, the maximum temperature an the final calcite composition.

In [43]:
//back calculated fluid composition from clumped temperature and bulk calcite oxygen composition
val d18OfluidBackCalculatedFun = (s:Step) => (d18OcalciteBulkFinal(s)*1.03092+30.92) - (18.03*(pow(10,3)/(sampleTempRecrystallised(s)+273.15))-32.42)


d18OfluidBackCalculatedFun: Step => Double = ammonite.$sess.cmd42$Helper$$Lambda$3337/0x00000008015b90b8@6fa80dbe

#### Modelling the potential temperature ogrnaic material has reached during burial. 

A simple first order arrehnius equation **(Burnham & Sweeney, 1989)**. 

For simplicity we has assumed that the freuqency factor and activation energy remain *constant*. Values taken from realistic values for the Eagle Ford and similar marine shales **(Burnham & Sweeney, 1989; Qigui et al., 2010)**.

In [44]:
//Reaction rate and Arrhenius equation
val reactionRateOrganicsFun = (s:Step) => math.pow(2, ((burialTemperature(s)-burialTemperature(s.i))/10))*reactionRateOrganics(s.i)

val apparentTempOrganicsFun = (s:Step) => Ea/((math.log(A)-math.log(reactionRateOrganics(s)))*R) - 273.15

reactionRateOrganicsFun: Step => Double = ammonite.$sess.cmd43$Helper$$Lambda$3341/0x00000008015ba500@16dc6ff
apparentTempOrganicsFun: Step => Double = ammonite.$sess.cmd43$Helper$$Lambda$3342/0x00000008015ba8d0@41226f91

#### Defining the model
The below model is calibrated to the hottest sample (EF316)

In [45]:
// Define the model here
val EagleFordModel = new SteppedModel(numberOfSteps,"Recrystallisation Model")
    .setGrid(19)
    .defineMathematicalModel(
      age =>> ageOfStep(initialAge,finalAge),
      depth =>> {(s:Step) => {burialDepthFromAgeModel(age,burialHistory).apply(s)+initalBurialAtModelStart(s.i)}},
      maxBurialDepth =>> {(s:Step) => {if(depth(s)>=depth(s-1)){depth(s)}else{depth(s-1)}}},
      surfaceTemperature =>> surfaceTemperaturesAtAge(age, surfaceTemperaturesHistory),
      geothermalGradient =>> geothermalGradientAtAge(age,geothermalGradientHistory),
      burialTemperature =>> burialTemperatureFromGeothermalGradient(surfaceTemperature,depth,geothermalGradient),
      dT =>> dTFun,
      reactionRateOrganics =>> reactionRateOrganicsFun,
      apparentTempOrganics =>> {(s:Step) => {if(burialTemperature(s)-burialTemperature(s-1)>0){apparentTempOrganicsFun(s)}else{apparentTempOrganics(s-1)}}},
      D47eq =>> {(s:Step) => (0.0390 * pow(10,6)/pow((burialTemperature(s)+273.15),2) + 0.154)},
      D47i =>> D47iFun,
      SampleTemp =>> {(s:Step) => (pow((0.0390 * pow(10, 6))/(D47eq(s)-0.154), 0.5)-273.15)}, //at equilibrium  
      fractionRecrystallisation =>> {(s:Step) => {if(burialTemperature(s)-burialTemperature(s-1)>0){fractionRecrystallisation(s)}else{0.0}}},
      weightFractionFluid =>> {(s:Step) => (Poro * rhofluid / (Poro * rhofluid + rhocal))},
      d18OfluidMixed =>> d18OfluidMixedFun,
      d18OfluidAfterDissolution =>> d18OfluidAfterDissolutionFun,
      d18OcalciteNewPhase =>> {(s:Step) => {if(fractionRecrystallisation(s)>0){d18OcalciteNewPhaseFun(s)}else{d18OcalciteNewPhase(s-1)}}},
      d18OfluidAfterReprecipitation =>> d18OfluidAfterReprecipitationFun,
      d18OcalciteBulkFinal =>> d18OcalciteBulkFinalFun,
      d13CfluidMixed =>> d13CfluidMixedFun,
      d13CfluidAfterDissolution =>> d13CfluidAfterDissolutionFun,
      d13CcalciteNewPhase =>> {(s:Step) => {if(fractionRecrystallisation(s)>0){d13CcalciteNewPhaseFun(s)}else{d13CcalciteNewPhase(s-1)}}},
      d13CfluidAfterReprecipitation =>> d13CfluidAfterReprecipitationFun,
      d13CcalciteBulkFinal =>> d13CcalciteBulkFinalFun,
      D47rec =>> {(s:Step) => (fractionRecrystallisation(s) * D47eq(s)) + ((1 - fractionRecrystallisation(s)) * D47rec(s-1))},
      sampleTempRecrystallised =>> {(s:Step) => (pow((0.0390 * pow(10, 6))/(D47rec(s)-0.154), 0.5)-273.15)}, //Anderson et al., 2021 in prep
      d18OfluidBackCalculated =>> d18OfluidBackCalculatedFun,
    ) 
    .defineInitialModelConditions(
      AllCells(initialAge,ageList),
      AllCells(finalAge,finalAgeList),
      AllCells(D47i, List(.59)),
      //AllCells(reactionRateOrganics, List(3.28E-26,3.28E-25,3.28E-24)),  
      PerCell(initalBurialAtModelStart,List(
        (List(-0.50),Seq(0)),
        (List(-0.91),Seq(1)),
        (List(-1.22),Seq(2)),
        (List(-2.90),Seq(3)),
        (List(-4.57),Seq(4)),
        (List(-4.88),Seq(5)),
        (List(-4.90),Seq(6)),
        (List(-6.10),Seq(7)),
        (List(-6.40),Seq(8)),
        (List(-6.71),Seq(9)),
        (List(-25.90),Seq(10)),
        (List(-28.96),Seq(11)),
        (List(-29.87),Seq(12)),
        (List(-35.05),Seq(13)),
        (List(-35.36),Seq(14)),
        (List(-35.97),Seq(15)),
        (List(-36.58),Seq(16)),
        (List(-37.19),Seq(17)),
        (List(-38.90),Seq(18)))),
    )
    .defineCalibration(
        D47rec.isEqualTo(0.426).atCells(Seq(4)),
        apparentTempOrganics.isBetween(45.0, 70.0).atCells(Seq(4)),
        //d18OfluidBackCalculated.isEqualTo(7.03).atCells(Seq(4)),
        //d18OcalciteBulkFinal.isEqualTo(-8.35).atCells(Seq(4)),
        //d13CcalciteBulkFinal.isEqualTo(0.33).atCells(Seq(4)),
    )                 

A total of 1 unique models were defined, attempting to create a list now.
Models list successfully created.


Feature,Value
Name,eaglefordModel4
Nb of steps,140
Nb of models,1
Nb grid cells,19
Nb of operations per step,27
Total nb of operations,71820


In [46]:
sim.evaluate(EagleFordModel)

Run progress: #################################################################################################### 100.0%

In [94]:
val results = sim.getResults(EagleFordModel)

eaglefordModel4 Contains 1 models each comprising 140 timesteps Model# Initial model conditions RSME 1 Initial age 97.0 at all cells Final age 0.0 at all cells Δ47i 0.59 at all cells Initial burial at model start -0.5 at cell (0), -0.91 at cell (1), -1.22 at cell (2), -2.9 at cell (3), -4.57 at cell (4), -4.88 at cell (5), -4.9 at cell (6), -6.1 at cell (7), -6.4 at cell (8), -6.71 at cell (9), -25.9 at cell (10), -28.96 at cell (11), -29.87 at cell (12), -35.05 at cell (13), -35.36 at cell (14), -35.97 at cell (15), -36.58 at cell (16), -37.19 at cell (17), -38.9 at cell (18), 0.003801525368053728